In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install sentencepiece
!pip install huggingface_hub
!pip install nltk

In [2]:
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric, Dataset
from datasets.dataset_dict import DatasetDict
from tqdm import tqdm
import pandas as pd
import ast
import gc
import torch
import collections
import numpy as np
from transformers import Trainer
from sklearn.utils import shuffle
import os
from accelerate import Accelerator
from transformers import get_scheduler
import math
from time import time
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    default_data_collator
)
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
def process_data(tokenizer, df_dir_path, df_name, max_len=256, truncation=True, pad_to_max_len=True):
  path = df_dir_path + df_name
  df = pd.read_csv(path)
  df.drop(columns=['Unnamed: 0'], inplace=True)
  d = {}
  input_ids = []
  attention_mask= []
  decoder_input_ids = []
  decoder_attention_mask = []
  for idx in tqdm(range(df.shape[0])):
    context, true_false, target = df.loc[idx, 'passage'], df.loc[idx, 'answer'], df.loc[idx, 'question']
    tokenizer_input = "answer: %s context: %s" % (true_false, context)
    tokenized_input = tokenizer(tokenizer_input, max_length=max_len, truncation=truncation, pad_to_max_length=pad_to_max_len, return_tensors='pt')
    tokenized_target = tokenizer(target, max_length=max_len, truncation=truncation, pad_to_max_length=pad_to_max_len, return_tensors='pt')
    input_ids.append(tokenized_input['input_ids'].squeeze())
    attention_mask.append(tokenized_input['attention_mask'].squeeze())
    decoder_input_ids.append(tokenized_target['input_ids'].squeeze())
    decoder_attention_mask.append(tokenized_target['attention_mask'].squeeze())
  d['input_ids'] = input_ids
  d['attention_mask'] = attention_mask
  d['decoder_input_ids'] = decoder_input_ids
  d['decoder_attention_mask'] = decoder_attention_mask
  dataset = Dataset.from_dict(d)
  return dataset

In [26]:
def get_mean_loss(dir_path, eval_csv, model_path, batch_size, tokenizer_name='t5-base'):
    tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)
    model = torch.load(model_path)
    eval_dataset = process_data(tokenizer, dir_path, eval_csv)
    data_collator = default_data_collator
    eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=data_collator)
    num_iterations = len(eval_dataset)//batch_size
    progress_bar_ = tqdm(range(num_iterations))
    device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'
    torch.cuda.empty_cache()
    model.to(device)
    mean_loss = 0
    num_batches = len(eval_dataset)//batch_size
    model.eval()
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
                batch = {k: v.to(device) for k, v in batch.items()}
                input_ids, attention_mask, labels, decoder_attention_mask = batch['input_ids'], batch['attention_mask'], batch['decoder_input_ids'], batch['decoder_attention_mask']
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask, labels=labels)
                loss = outputs.loss
                mean_loss += loss
    mean_loss/= num_batches
    return mean_loss

In [27]:
dir_path = "/home/jovyan/PlanetaryComputerExamples/AI Planet Task/dataset/"
eval_csv = "val_df.csv"
batch_size = 4

In [19]:
model_path = '/home/jovyan/PlanetaryComputerExamples/AI Planet Task/t5-base-boolean-qgen_pretrained-finetuned.bin'
mean_loss = get_mean_loss(dir_path, eval_csv, model_path, batch_size)
print("t5 base pretrained qgen finetuned eval mean loss: ", mean_loss)

/srv/conda/envs/notebook/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/3270 [00:00<?, ?it/s]/srv/conda/envs/notebook/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future 

t5 base pretrained qgen finetuned eval mean loss:  tensor(0.0631, device='cuda:0')


In [20]:
model_path = '/home/jovyan/PlanetaryComputerExamples/AI Planet Task/t5-base-finetuned.bin'
mean_loss2 = get_mean_loss(dir_path, eval_csv, model_path, batch_size)
print("t5 base finetuned eval mean loss: ", mean_loss)

/srv/conda/envs/notebook/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/3270 [00:00<?, ?it/s]/srv/conda/envs/notebook/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future 

t5 base finetuned eval mean loss:  tensor(0.0631, device='cuda:0')


In [30]:
model_path = '/home/jovyan/PlanetaryComputerExamples/AI Planet Task/t5-small_5epochs.bin'
mean_loss3 = get_mean_loss(dir_path, eval_csv, model_path, batch_size, tokenizer_name='t5-small')
print("t5 small finetuned eval mean loss: ", mean_loss)



  0%|          | 0/3270 [00:00<?, ?it/s]/srv/conda/envs/notebook/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  4%|▍         | 126/3270 [00:00<00:02, 1259.08it/s]

  8%|▊         | 252/3270 [00:00<00:02, 1252.11it/s]

 12%|█▏        | 379/3270 [00:00<00:02, 1257.90it/s]

 16%|█▌        | 512/3270 [00:00<00:02, 1281.35it/s]

 20%|█▉        | 641/3270 [00:00<00:02, 1278.55it/s]

 24%|██▎       | 773/3270 [00:00<00:01, 1292.40it/s]

 28%|██▊       | 903/3270 [00:00<00:01, 1288.61it/s]

 32%|███▏      | 1032/3270 [00:00<00:

t5 base finetuned eval mean loss:  tensor(0.0631, device='cuda:0')
